## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sumenep,ID,-7.0167,113.8667,84.67,75,91,14.36,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,82.22,71,95,3.89,overcast clouds
2,2,Bluff,NZ,-46.6000,168.3333,47.80,69,94,22.24,overcast clouds
3,3,Hilo,US,19.7297,-155.0900,78.53,77,75,5.75,broken clouds
4,4,Dikson,RU,73.5069,80.5464,35.35,87,76,6.69,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sumenep,ID,-7.0167,113.8667,84.67,75,91,14.36,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,82.22,71,95,3.89,overcast clouds
3,3,Hilo,US,19.7297,-155.0900,78.53,77,75,5.75,broken clouds
5,5,Hamilton,US,39.1834,-84.5333,83.73,75,0,5.75,clear sky
10,10,Tautira,PF,-17.7333,-149.1500,78.87,79,26,16.93,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                222
City                   222
Country                222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sumenep,ID,84.67,overcast clouds,-7.0167,113.8667,
1,Hithadhoo,MV,82.22,overcast clouds,-0.6000,73.0833,
3,Hilo,US,78.53,broken clouds,19.7297,-155.0900,
5,Hamilton,US,83.73,clear sky,39.1834,-84.5333,
10,Tautira,PF,78.87,light rain,-17.7333,-149.1500,
11,Avarua,CK,77.05,clear sky,-21.2078,-159.7750,
13,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,
15,Cairns,AU,84.11,scattered clouds,-16.9167,145.7667,
23,Jamestown,US,76.05,scattered clouds,42.0970,-79.2353,
25,Pundaguitan,PH,82.45,moderate rain,6.3711,126.1689,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found . . . skipping.")
    
        

Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.
Hotel not found . . . skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.notnull().sum()

City                   222
Country                222
Max Temp               222
Current Description    222
Lat                    222
Lng                    222
Hotel Name             222
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Ccountry</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.15)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))